### Test status indication in jupyter notebook
This will be a brief demonstration of the indicator tools for jupyter notebook. The indicators in `labbench` include
* `panel`, which autogenerates a table of state values, and
* `log_progress`, which shows through the test mainloop.
* `linspace(...)` is a shortcut for `log_progress(np.linspace(...))`
* `range(...)` is a shortcut for `log_progress(range(...))`

`panel` is called automatically from within a notebook on calls to `log_progress`, `linspace`, or `range`.

To get started here is the emulated instrument setup.

In [1]:
import sys,os
if '..' not in sys.path:
    sys.path.insert(0,'..')
import labbench as lb
from labbench.notebooks import panel

import numpy as np
import pandas as pd
from time import strftime

from sim_visa import PowerSupply, SpectrumAnalyzer

Here is the example use of the indicators. They are all triggered automatically by the use of `lb.linspace`. The explicit call to `panel()` is also included.

`panel` does introspection on the code where it is called to identify `lb.Device` instances. It observes changes to the states of these devices and shows a continuously updating table as the test progresses.

`log_progress` shows a bar indicating the progress through a test loop.

In [5]:
import numpy as np
from labbench.notebooks import panel
from tqdm.notebook import tqdm as progress
import time
lb._force_full_traceback(True)

with SpectrumAnalyzer('GPIB::15::INSTR') as analyzer,\
     PowerSupply('USB::0x1111::0x2222::0x2468::INSTR') as supply,\
     lb.CSVLogger(path=f"data-{strftime('%Y-%m-%d %Hh%Mm%Ss')}") as db:

    panel()

    # include trait changes from all instruments in the database
    db.observe([analyzer, supply])

    # get on every row
    db.observe(analyzer, always='sweeps')

    for analyzer.frequency in progress(np.linspace(10e6,100e6,31), desc='frequency'):
        supply.voltage = 5.38
        
        db.new_row(
            spectrum=analyzer.fetch_trace(),
            current_draw=supply.fetch_trace(),
            power_gW=1.21e9
        )

    db.write()

2021-05-21 14:38:10.579 - INFO - CSVLogger('data-2021-05-21 14h38m10s') - context entry 0.501 s elapsed


AttributeError: 'frame' object has no attribute 'frame'

In [3]:
db.__enter__

<bound method RelationalTableLogger.__enter__ of CSVLogger('data-2021-05-21 13h23m16s')>

In [4]:
import json
json.dump?

Signature:
json.dump(
    obj,
    fp,
    *,
    skipkeys=False,
    ensure_ascii=True,
    check_circular=True,
    allow_nan=True,
    cls=None,
    indent=None,
    separators=None,
    default=None,
    sort_keys=False,
    **kw,
)
Docstring:
Serialize ``obj`` as a JSON formatted stream to ``fp`` (a
``.write()``-supporting file-like object).

If ``skipkeys`` is true then ``dict`` keys that are not basic types
(``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
instead of raising a ``TypeError``.

If ``ensure_ascii`` is false, then the strings written to ``fp`` can
contain non-ASCII characters if they appear in strings contained in
``obj``. Otherwise, all such characters are escaped in JSON strings.

If ``check_circular`` is false, then the circular reference check
for container types will be skipped and a circular reference will
result in an ``OverflowError`` (or worse).

If ``allow_nan`` is false, then it will be a ``ValueError`` to
serialize out of range ``float`` 

In [3]:
db.host.log

TypeError: 'DisconnectedBackend' object is not subscriptable

In [3]:
import yaml
import json

from datetime import date, datetime

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

d = yaml.load(log)

print(json.dumps(d, default=json_serial, indent=True))

<ipython-input-3-f697d82f10f4>:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  d = yaml.load(log)


ParserError: expected '<document start>', but found '{'
  in "<unicode string>", line 6, column 1:
    {
    ^

In [18]:
d

[{'message': 'new data row has 8 columns',
  'time': datetime.datetime(2021, 5, 21, 12, 29, 43, 449929),
  'level': 'DEBUG'},
 {'message': 'write "\':FREQ 100000000.0\'"',
  'time': datetime.datetime(2021, 5, 21, 12, 29, 43, 450930),
  'level': 'DEBUG',
  'device': 'SpectrumAnalyzer()'},
 {'message': 'write "\':VOLT:IMM:AMPL 5.38\'"',
  'time': datetime.datetime(2021, 5, 21, 12, 29, 43, 451929),
  'level': 'DEBUG',
  'device': 'PowerSupply()'}]

In [16]:
json.dumps?

Signature:
json.dumps(
    obj,
    *,
    skipkeys=False,
    ensure_ascii=True,
    check_circular=True,
    allow_nan=True,
    cls=None,
    indent=None,
    separators=None,
    default=None,
    sort_keys=False,
    **kw,
)
Docstring:
Serialize ``obj`` to a JSON formatted ``str``.

If ``skipkeys`` is true then ``dict`` keys that are not basic types
(``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
instead of raising a ``TypeError``.

If ``ensure_ascii`` is false, then the return value can contain non-ASCII
characters if they appear in strings contained in ``obj``. Otherwise, all
such characters are escaped in JSON strings.

If ``check_circular`` is false, then the circular reference check
for container types will be skipped and a circular reference will
result in an ``OverflowError`` (or worse).

If ``allow_nan`` is false, then it will be a ``ValueError`` to
serialize out of range ``float`` values (``nan``, ``inf``, ``-inf``) in
strict compliance of the JSON speci

In [13]:
db.observe?

Signature: db.observe(devices, changes=True, always=[], never=['isopen'])
Docstring:
Configure the data to aggregate from value, property, or datareturn traits in the given devices.

Device may be a single device instance, or an
several devices in an iterable (such as a list
or tuple) to apply to each one.

Subsequent calls to :func:`observe_states` replace
the existing list of observed property traits for each
device.

Args:
    devices: Device instance or iterable of Device instances
    changes (bool): Whether to automatically log each time a property trait is set for the supplied device(s)

    always: name (or iterable of multiple names) of property traits to actively update on each call to get()
    never: name (or iterable of multiple names) of property traits to exclude from aggregated result (overrides :param:`always`)
File:      c:\users\dkuester\documents\src\labbench\labbench\_data.py
Type:      method


In [5]:
huh = tqdm(desc='hi')